# Binary puzzle/ Binario/ Takuzu puzzle
## GPU bruteforce puzzle generator (xD)
https://en.wikipedia.org/wiki/Takuzu

Rules:
1. Each row or column should have equal number of 1s and 0s
1. Cannot have more than 2 grouped entries horizontally or vertically

Approach:
1) Is checked via sum on proper axis
2) Is checked via convolution/cross-correlation

In [37]:
import torch.nn.functional as F
import torch
from misc_tools.print_latex import print_tex
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32
dtype0 = int
puzz = torch.tensor([   [1,0,1,0,1,0],
                        [0,1,0,0,1,1],
                        [1,0,0,1,0,1],
                        [0,1,1,0,1,0],
                        [0,0,1,1,0,1],
                        [1,1,0,1,0,0]], 
                        device=device,
                        dtype = dtype).unsqueeze(0).unsqueeze(0)

# img = puzz.repeat_interleave(2,0)
# img[0,0,0,1] = 1

ker_v  = torch.tensor([ [0,1,0],
                        [0,1,0],
                        [0,1,0]], 
                        dtype=dtype, 
                        device=device).unsqueeze(0).unsqueeze(0)

ker_h = torch.tensor([  [0,0,0],
                        [1,1,1],
                        [0,0,0]], 
                        dtype=dtype, 
                        device=device).unsqueeze(0).unsqueeze(0)


gpu_conv = lambda img, ker: F.conv2d(img, ker, bias=None, stride=1, padding='same', dilation=1, groups=1).to(device)

def puzzle_fails_GPU(puzz):
    # dims [batch, 1, N, N]; apply (1,1,3,3) convolution
    batch, _, N, _ = puzz.shape
    D = N//2
    # count 1s in rows & cols by summing along according dimensions. ?= D -> [batch, 1, N]
    # combine tests for rows and columns. we want to see if any batch 'not fails' :O
    num_elem_fails = (torch.sum(puzz, axis = -1) != D) | (torch.sum(puzz, axis = -2) != D)
    # test if some row/colum fails batch wise (along N dim). invert to see if did not fail
    #print(num_elem_fails)
    # ~fail = pass
    num_elem_pass = ~torch.any(num_elem_fails, dim=-1).flatten()

    three_neighbor_pass =  ~((gpu_conv(  puzz, ker_v) == 3) |   # check vert & horiz neighbors 
                             (gpu_conv(  puzz, ker_h) == 3) |   # of ones and zeros
                             (gpu_conv(1-puzz, ker_v) == 3) |   # cannot invert float 
                             (gpu_conv(1-puzz, ker_h) == 3) )   # 1-1 = 0, 1-0 = 1
    
    # flatten each batch so i can use torch.all() pass
    three_neighbor_pass = three_neighbor_pass.view(batch, 1, -1)
    
    three_neighbor_pass2 = torch.all(three_neighbor_pass, dim = -1).flatten()
    return num_elem_pass & three_neighbor_pass2

print_tex(puzz[0,0].cpu().numpy())
print(f'Puzzle is correct: {puzzle_fails_GPU(puzz).item()}')


<IPython.core.display.Math object>

Puzzle is correct: True


In [32]:
# generate puzzles by filling ~1/2 with ones.
from tqdm import tqdm
N = 6
B = 200000
for i in tqdm(range(10000)):
    #torch.manual_seed(i)
    batch = (torch.rand(size=(B,1,N,N), device=device) > 0.5).to(dtype)
    res = puzzle_fails_GPU(batch)
    if torch.any(res).item():
        sol_id = torch.argwhere(res).flatten()[0]
        print(f'{i = }. Checked {i*B:0.1e} puzzles in total')
        print_tex(batch[sol_id].reshape(N,N).cpu().numpy())
        break

  0%|          | 23/10000 [00:00<03:48, 43.65it/s]

i = 26. Checked 5.2e+06 puzzles in total


<IPython.core.display.Math object>

  0%|          | 26/10000 [00:00<04:51, 34.26it/s]
